# Evaluation - Notebook

Within this document, we carry out an assessment of the implemented models. Furthermore, hyperparameter optimization is executed for each model wherever feasible. The following models are subjected to evaluation:

1. Baseline
2. Moving Average
3. Various variants of linear regression
4. Neural Network
5. XGBoost
6. LSTM

For a more detailed description of the models, please refeir to their corresponding section.

In [1]:
# TODO: look into pipeline framework for hyperparameter tuning
# ###: 1. try different window sizes for the models (where possible) (Part of hyperparameter tuning)
# ###: 2. discount rate for moving average
# ###: 3. alpha for lasso and ridge regression
# ###: 4. polynomial features for linear regression 
# ###: 5. Different Features (difficult)
# TODO: Rewrite loading.py file
# TODO: try the transfer learning approach
# TODO: implement command line interface
# TODO: implement logging
# TODO: Documentation 
# TODO: Hand-in

In [2]:
# imports
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import TimeSeriesSplit



import sys

sys.path.append("..")
sys.path.append("../src")

# from src.DataHandling.processing import supervised_transform
import src.DataHandling.visualization as vis

# models
from src.Models.ma import MovingAverage
from src.Models.lr import Regression

# Transformers
from src.DataHandling.preprocessing import (
    # DataCleaner,
    SupervisedTransformer,
    train_test_split,
)

# hyperparameter tuning
from src.Models.selection import GridSearch

In [3]:
# load cleaned data
turbine_brit = pd.read_csv(f"../data/cleaned/turbine_brit_{2}.csv")
turbine_braz = pd.read_csv(f"../data/cleaned/turbine_braz_{1}.csv")

# use date column as index and convert to datetime
turbine_brit["Date"] = pd.to_datetime(turbine_brit["Date"])
turbine_brit.set_index("Date", inplace=True)

turbine_braz["Date"] = pd.to_datetime(turbine_braz["Date"])
turbine_braz.set_index("Date", inplace=True)

DATA = {"British": turbine_brit, "Brazilian": turbine_braz}

For understandibility.... models do transformation to a supervised learning problem inherently...

Before, we begin blablabla ... their are shared parameters across models like window_size etc.

In [4]:
benchmarks = pd.read_csv("../results_wind.csv")
test_start = benchmarks["test_start"][0]
test_end = benchmarks["test_end"][0]


X_train, y_train, X_test, y_test = train_test_split(
    test_start=test_start, test_end=test_end, df=turbine_brit, target_var="Power (kW)"
)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((233606, 2), (233606,), (52560, 2), (52560,))

In [5]:
from collections import OrderedDict

param_grid = {
    "st__window_size": [1, 5, 10],
    "st__horizon": [1, 6, 144],
}

index = pd.MultiIndex.from_product(iterables=list(param_grid.values()), names=list(param_grid.keys()))
results = pd.DataFrame(index=index, columns=["mae", "rmse"]).sort_index()
results


mae rmse
st__window_size st__horizon          
1               1            NaN  NaN
                6            NaN  NaN
                144          NaN  NaN
5               1            NaN  NaN
                6            NaN  NaN
                144          NaN  NaN
10              1            NaN  NaN
                6            NaN  NaN
                144          NaN  NaN

## 1. Moving Average

**Explanation of the Model:** 

In contrast to the Baseline model, the Moving Average model offers the flexibility of selecting a window size. This window size determines the number of preceding time steps taken into account for generating forecasts. Furthermore, there is an option to specify a discount factor (default = 1), which regulates the extent of reduction in influence for more distant past time steps. This strategy is similar to how a discount factor is used in calculating future rewards within the framework of reinforcement learning. Note that when a window size of one is chosen, the Moving Average model is identical with the Baseline model.

In [6]:
ma_pipe = Pipeline(
    [
        ("st", SupervisedTransformer()),
        ("Model", MovingAverage()),
    ]
)

param_grid["Model__discount"] = np.linspace(0.5, 1., 3)

grid_search_ma = GridSearch(ma_pipe, param_grid)
grid_search_ma.fit(X_train, y_train, X_test, y_test)

In [13]:
grid_search_ma.results

RMSE         MAE
Model__discount st__horizon st__window_size                        
0.50            1           1                148.755907   48.093292
                            5                152.187079   53.309690
                            10               152.384043   53.602425
                6           1                269.454486  100.944792
                            5                260.798651   99.370914
                            10               260.125004   99.174378
                144         1                751.241013  373.817298
                            5                745.496040  371.952560
                            10               744.822448  371.654857
0.75            1           1                148.755907   48.093292
                            5                164.392840   59.002228
                            10               171.065713   63.475630
                6           1                269.454486  100.944792
                            5                262.359078  100.633266
                            10               262.583636  101.885182
                144         1                751.241013  373.817298
                            5                744.196466  372.338477
                            10               741.566608  371.972988
1.00            1           1                148.755907   48.093292
                            5                179.454348   65.206409
                            10               207.512206   80.199938
                6           1                269.454486  100.944792
                            5                267.502014  103.144049
                            10               279.079979  109.225920
                144         1                751.241013  373.817298
                            5                744.359058  372.812025
                            10               741.716070  372.186491

In [8]:
# delete key of param_grid that is specific to the model
del param_grid["Model__discount"]
param_grid

{'st__window_size': [1, 5, 10], 'st__horizon': [1, 6, 144]}

## 2. Different kinds of Linear Regression

In [9]:
lr_pipe = Pipeline(
    [
        ("st", SupervisedTransformer()),
        ("Model", Regression()),
    ]
)

param_grid["Model__model"] = ["linear", "ridge"]
param_grid["Model__alpha"] = np.logspace(-3, 1, 5)

In [10]:
param_grid["Model__alpha"]

array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01])

In [11]:
grid_search_lr = GridSearch(lr_pipe, param_grid)
grid_search_lr.fit(X_train, y_train, X_test, y_test)
grid_search_lr.best_params

{'Model__alpha': 0.001,
 'Model__model': 'ridge',
 'st__horizon': 1,
 'st__window_size': 10}

In [12]:
grid_search_lr.results

RMSE         MAE
Model__alpha Model__model st__horizon st__window_size                        
0.001        linear       1           1                147.639457   52.476637
                                      5                144.636953   51.114800
                                      10               144.366104   51.115401
                          6           1                263.345871  124.328627
                                      5                255.621265  116.026479
...                                                           ...         ...
10.000       ridge        6           5                255.621264  116.026487
                                      10               254.749978  114.140046
                          144         1                622.409722  438.181487
                                      5                621.785133  436.094431
                                      10               621.716477  435.319673

[90 rows x 2 columns]